In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.python.keras import models, layers, optimizers
import tensorflow
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
import bz2
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score
import re

%matplotlib inline

In [ ]:
import gzip

# with gzip.open('dataset/amazon_reviews_us_Books_v1_02.tsv.gz','r') as fin:        
#     for line in fin:        
#         print('got line', line)

In [ ]:
# import gzip
# import io
# gz = gzip.open('dataset/amazon_reviews_us_Watches_v1_00.tsv.gz', 'rb')
# f = io.BufferedReader(gz)
# counter = 0

# for line in f.readlines():
#     print(line)
#     if counter > 20:
#         break
#     counter=counter+1
            
# gz.close()

In [ ]:
import pandas as pd

# df = pd.read_csv('dataset/amazon_reviews_us_Watches_v1_00.tsv.gz', sep='\t', header=0)
df = pd.read_csv('dataset/amazon_reviews_us_Watches_v1_00.tsv', sep='\t', error_bad_lines=False)

In [ ]:
df.head(4)
# df.vine.value_counts()


In [ ]:
df[df['star_rating'] > 3]

In [ ]:
df['review_body'][df['star_rating'] > 3]

In [61]:
df.isna().sum()

marketplace          0
customer_id          0
review_id            0
product_id           0
product_parent       0
product_title        0
product_category     0
star_rating          0
helpful_votes        0
total_votes          0
vine                 0
verified_purchase    0
review_headline      0
review_body          0
review_date          0
all_review           0
clean_all_review     0
dtype: int64

In [60]:
df = df.where(pd.notnull(df), "")

### Combine Review Body & Head

In [63]:
df['all_review'] = df['review_headline']+" "+df['review_body']

In [ ]:
# Unused

# emoticon_repl = {
#     # positive emoticons
#     r":-?d+": " good ", # :D, :-D, :DD, ecc. in lower case
#     r":[- ]?\)+": " good ", # :-), :), :-)), :)), ecc
#     r";-?\)+": " good ", # ;). ;)). ;-), ecc.
#     r"\(+-?:": " good ", # (:, (-:, ecc.
#     r"=\)+" : " good ",
#     r"\b<3\b" : " good ",    
#     # negative emoticons
#     r"[\s\r\t\n]+:/+": " bad ", # :/
#     r":\\+": " bad ", # :\
#     r"[\s\r\t\n]+\)-?:": " bad ",  # ):, )):, )-:, ecc.
#     r":-?\(+": " bad ", # :(, :-(, :((, ecc.
#     r"[\s\t\r\n]+d+-?:": " bad "
# }

In [9]:
from collections import defaultdict
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.corpus import wordnet as wn
from nltk.tokenize import word_tokenize

tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [88]:
def preprocess(sent):
    
    sent = str(sent).lower()
    sent = re.sub(r'^<div id="video.*>&nbsp;', '', sent) # Video-review part
    sent = re.sub('https?://[A-Za-z0-9./]+', '', sent) # URLs
    
    # Remove HTML Tag
    sent = re.sub('<.*?>', '', sent)    
    
    # Remove everything in bracket    
    sent = re.sub('\[[^)]*\]', '', sent)    
    
#     for k in emoticon_repl:
#         sent = re.sub(k, emoticon_repl[k], sent)
    
    sent = re.sub('[/]+', ' ', sent) # word1/word2 to word1 word2
    # Remove non-alphanumeric characters (but not - and _, might be useful for tokenization)
    sent = re.sub('[^A-Za-z0-9-_ ]+', '', sent)

    # Remove words that are digits only
    sent = re.sub('\b\d+\b', '', sent)                
    
    return sent

###  Apply Preprocess

In [99]:
df['clean_all_review'] = df['clean_all_review'].apply(preprocess)

In [100]:
df.head(5)

marketplace  customer_id       review_id  product_id  product_parent  \
0          US      3653882  R3O9SGZBVQBV76  B00FALQ1ZC       937001370   
1          US     14661224   RKH8BNC3L5DLF  B00D3RGO20       484010722   
2          US     27324930  R2HLE8WKZSU3NL  B00DKYC7TK       361166390   
3          US      7211452  R31U3UH5AZ42LL  B000EQS1JW       958035625   
4          US     12733322  R2SV659OUJ945Y  B00A6GFD7S       765328221   

                                       product_title product_category  \
0  Invicta Women's 15150 "Angel" 18k Yellow Gold ...          Watches   
1  Kenneth Cole New York Women's KC4944 Automatic...          Watches   
2  Ritche 22mm Black Stainless Steel Bracelet Wat...          Watches   
3  Citizen Men's BM8180-03E Eco-Drive Stainless S...          Watches   
4  Orient ER27009B Men's Symphony Automatic Stain...          Watches   

   star_rating  helpful_votes  total_votes vine verified_purchase  \
0            5              0            0    N                 Y   
1            5              0            0    N                 Y   
2            2              1            1    N                 Y   
3            5              0            0    N                 Y   
4            4              0            0    N                 Y   

                              review_headline  \
0                                  Five Stars   
1  I love thiswatch it keeps time wonderfully   
2                                   Two Stars   
3                                  Five Stars   
4    Beautiful face, but cheap sounding links   

                                         review_body review_date  \
0  Absolutely love this watch! Get compliments al...  2015-08-31   
1       I love this watch it keeps time wonderfully.  2015-08-31   
2                                          Scratches  2015-08-31   
3  It works well on me. However, I found cheaper ...  2015-08-31   
4  Beautiful watch face.  The band looks nice all...  2015-08-31   

                                          all_review  \
0  five stars absolutely love this watch get comp...   
1  i love thiswatch it keeps time wonderfully i l...   
2                                two stars scratches   
3  five stars it works well on me however i found...   
4  beautiful face but cheap sounding links beauti...   

                                    clean_all_review  
0  five stars absolutely love watch get complimen...  
1  love thiswatch keeps time wonderfully love wat...  
2                                two stars scratches  
3  five stars works well however found cheaper pr...  
4  beautiful face cheap sounding links beautiful ...

### Remove Stopwords

In [11]:
len_df = len(df)
# print("%.7f" % p +" yeye")

In [27]:
# stopwords
# WARNING: TAKES LONG TIME TO PROCESS

from IPython.display import display, clear_output
import time

start = 960203
end   = 960205
t1 = time.time()
for entry in df['all_review'][start:end]:    
    clear_output(wait=True)
    res = [word for word in word_tokenize(entry) if not word in stopwords.words('english')]    
    res = (" ").join(res)  
    p = (start/len_df)*100
    display("%.7f" % p +" - "+str(start))
    df.loc[start,'clean_all_review'] = res
    if start >= end:
        break
    start = start+1
    
df.to_csv('amazon_reviews_us_Watches_vcleanv03.csv')
t2 = time.time()
print('Duration: {0:.2f}s'.format(round(t2-t1,2)))

'99.9998959 - 960203'

Duration: 37.53s


500 loop - Duration: 76.76s

In [25]:
df.head(5)

marketplace  customer_id       review_id  product_id  product_parent  \
0          US      3653882  R3O9SGZBVQBV76  B00FALQ1ZC       937001370   
1          US     14661224   RKH8BNC3L5DLF  B00D3RGO20       484010722   
2          US     27324930  R2HLE8WKZSU3NL  B00DKYC7TK       361166390   
3          US      7211452  R31U3UH5AZ42LL  B000EQS1JW       958035625   
4          US     12733322  R2SV659OUJ945Y  B00A6GFD7S       765328221   

                                       product_title product_category  \
0  Invicta Women's 15150 "Angel" 18k Yellow Gold ...          Watches   
1  Kenneth Cole New York Women's KC4944 Automatic...          Watches   
2  Ritche 22mm Black Stainless Steel Bracelet Wat...          Watches   
3  Citizen Men's BM8180-03E Eco-Drive Stainless S...          Watches   
4  Orient ER27009B Men's Symphony Automatic Stain...          Watches   

   star_rating  helpful_votes  total_votes vine verified_purchase  \
0            5              0            0    N                 Y   
1            5              0            0    N                 Y   
2            2              1            1    N                 Y   
3            5              0            0    N                 Y   
4            4              0            0    N                 Y   

                              review_headline  \
0                                  Five Stars   
1  I love thiswatch it keeps time wonderfully   
2                                   Two Stars   
3                                  Five Stars   
4    Beautiful face, but cheap sounding links   

                                         review_body review_date  \
0  Absolutely love this watch! Get compliments al...  2015-08-31   
1       I love this watch it keeps time wonderfully.  2015-08-31   
2                                          Scratches  2015-08-31   
3  It works well on me. However, I found cheaper ...  2015-08-31   
4  Beautiful watch face.  The band looks nice all...  2015-08-31   

                                          all_review  \
0  five stars absolutely love this watch get comp...   
1  i love thiswatch it keeps time wonderfully i l...   
2                                two stars scratches   
3  five stars it works well on me however i found...   
4  beautiful face but cheap sounding links beauti...   

                                    clean_all_review  
0  five stars absolutely love watch get complimen...  
1  love thiswatch keeps time wonderfully love wat...  
2                                two stars scratches  
3  five stars works well however found cheaper pr...  
4  beautiful face cheap sounding links beautiful ...

In [26]:
# df.to_csv('amazon_reviews_us_Watches_vcleanv03.csv')
df[['all_review','clean_all_review']][start-8:start+5]

# df[497387-5:497387+5]
# df[['all_review','clean_all_review']][746000-5:746000+5]
# df.iloc[497388]

all_review  \
960196  it takes a licking ive been very pleased with ...   
960197  adorable but fragile the 13 year old girl i bo...   
960198  great for interval training this hrm is pretty...   
960199  great prize - but size its a great watch - but...   
960200  a very classy watch well worth the price this ...   
960201  this is a great watch dear targeteersthis watc...   
960202  now watt a minute here in the old days the com...   
960203  1 10 watt difference for hearing aids i have f...   

                                         clean_all_review  
960196  takes licking ive pleased new timex expedition...  
960197  adorable fragile 13 year old girl bought loves...  
960198  great interval training hrm pretty darn accura...  
960199  great prize - size great watch - dont chance c...  
960200  classy watch well worth price light weight cla...  
960201  great watch dear targeteersthis watch exelent ...  
960202  watt minute old days common hearing battery pa...  
960203  1 10 watt difference hearing aids found 393 wa...

### Reload Preprocessed Data

In [2]:
df = pd.read_csv('amazon_reviews_us_Watches_vcleanv03.csv', index_col=0, low_memory=False)

In [3]:
df.head(5)

marketplace  customer_id       review_id  product_id  product_parent  \
0          US      3653882  R3O9SGZBVQBV76  B00FALQ1ZC       937001370   
1          US     14661224   RKH8BNC3L5DLF  B00D3RGO20       484010722   
2          US     27324930  R2HLE8WKZSU3NL  B00DKYC7TK       361166390   
3          US      7211452  R31U3UH5AZ42LL  B000EQS1JW       958035625   
4          US     12733322  R2SV659OUJ945Y  B00A6GFD7S       765328221   

                                       product_title product_category  \
0  Invicta Women's 15150 "Angel" 18k Yellow Gold ...          Watches   
1  Kenneth Cole New York Women's KC4944 Automatic...          Watches   
2  Ritche 22mm Black Stainless Steel Bracelet Wat...          Watches   
3  Citizen Men's BM8180-03E Eco-Drive Stainless S...          Watches   
4  Orient ER27009B Men's Symphony Automatic Stain...          Watches   

   star_rating  helpful_votes  total_votes vine verified_purchase  \
0            5              0            0    N                 Y   
1            5              0            0    N                 Y   
2            2              1            1    N                 Y   
3            5              0            0    N                 Y   
4            4              0            0    N                 Y   

                              review_headline  \
0                                  Five Stars   
1  I love thiswatch it keeps time wonderfully   
2                                   Two Stars   
3                                  Five Stars   
4    Beautiful face, but cheap sounding links   

                                         review_body review_date  \
0  Absolutely love this watch! Get compliments al...  2015-08-31   
1       I love this watch it keeps time wonderfully.  2015-08-31   
2                                          Scratches  2015-08-31   
3  It works well on me. However, I found cheaper ...  2015-08-31   
4  Beautiful watch face.  The band looks nice all...  2015-08-31   

                                          all_review  \
0  five stars absolutely love this watch get comp...   
1  i love thiswatch it keeps time wonderfully i l...   
2                                two stars scratches   
3  five stars it works well on me however i found...   
4  beautiful face but cheap sounding links beauti...   

                                    clean_all_review  
0  five stars absolutely love watch get complimen...  
1  love thiswatch keeps time wonderfully love wat...  
2                                two stars scratches  
3  five stars works well however found cheaper pr...  
4  beautiful face cheap sounding links beautiful ...

In [ ]:
X=df.all_review.values
y=df.star_rating > 3

def check_presence(regex, stop = 1, verbose = True):
    j = 0
    tv = False
    for i in range(len(X)):
        if re.search(regex, str(X[i]).lower()):
            tv = True
            j += 1
            sentiment = "positive" if y[i] else "negative"
            if verbose:
                print("Review #",i)
                print("Sentiment: {}".format(sentiment))
                print("Occurrences of the regex {} :".format(regex), re.findall(regex, X[i].lower()))
                print("Review text:", X[i]+"\n")
            if j == stop:
                break
    return tv

In [ ]:
check_presence(r":-?d+")

In [ ]:
preprocess(X[1249])
# X[1249]

### Set Positive Sentiments

In [ ]:
df["positive"] = df.star_rating > 3

# Temp

In [107]:
# X[1249]
print(df['review_body'][1249])
print("")
print(df['all_review'][1249])
# print(df['clean_all_review'][1249])

[[VIDEOID:dae27737f337c776423c7999c6781874]]This BUREI watch is an attractive quartz movement watch. The watch is classy and elegant. It has a stainless steel body and genuine leather band.  The band maximum width 8.25\\" (last hole), if needed additional holes could be made to make it as wide as 8.5\\".<br /><br />The watch's face is 1.5\\" wide and the watch is 3/8\\" thick.<br /><br />This watch is made in China but includes an English manual. The manual covers several watches so you need to match your dial to one in the photo to get the details on how to set various options.<br /><br />The first position of the crown allows to adjust days of the month.<br />The second position of the crown allows to change time.<br />To select the day of the week I need to rotate the time until the right day appears in the display window.  This is a bit time consuming as to go to the next day you need to rotate through 24 hours.  If you miss it you cannot go backwards to you can only go forward.<br

In [ ]:
df['total_votes'].value_counts()

In [ ]:
df = df.drop('clean_all_review', axis=1)

In [ ]:
df